# License Plate Detection using an Improved FPN Architecture

This notebook implements an enhanced Feature Pyramid Network (FPN) for license plate detection with specific focus on:

1. Improving size estimation accuracy
2. Better detection of small license plates
3. Using advanced attention mechanisms
4. Implementing multi-scale feature detection

The code uses our refactored license plate detection package with modular components.

# Step 1: Data Loading and Exploration
We'll load and visualize the dataset to understand our license plate detection task.

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
import sys
import random

# Check if running in Colab
import importlib.util
IN_COLAB = importlib.util.find_spec("google.colab") is not None

current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

if IN_COLAB:
    # Add project root to path to ensure imports work correctly
    project_root = os.path.join(current_dir, "Car-plate-detection")
    sys.path.insert(0, project_root)
    print(f"Project root added to path: {project_root}")
    DATA_PATH = Path(project_root+"/Dataset")
else:
    # If not in Colab, set the project root to the current working directory's parent
    project_root = Path(os.getcwd()).parent
    print(f"Project root: {project_root}")
    # Add project root to path to fix import errors
    sys.path.insert(0, str(project_root))
    print(f"Project root added to path: {project_root}")
    DATA_PATH = project_root / "Dataset"

# Print TensorFlow version for reference
print(f"TensorFlow version: {tf.__version__}")

# Import modules from license_plate_detection package
from license_plate_detection.models.losses import enhanced_iou_metric, improved_combined_detection_loss, giou_loss
from license_plate_detection.models.fpn_detector import create_fpn_license_plate_detector
from license_plate_detection.data.loader import get_data_path, load_license_plate_dataset, preprocess_license_plate_dataset, split_dataset
from license_plate_detection.data.augmentation import augment_data, visualize_augmentation
from license_plate_detection.train.trainer import train_model, save_model, train_model_with_datasets, create_efficient_data_pipeline
from license_plate_detection.train.scheduler import create_lr_scheduler
from license_plate_detection.utils.memory_optimizations import optimize_memory_usage, enable_gradient_checkpointing, clean_memory, setup_gpu_memory_growth, limit_gpu_memory, enable_mixed_precision
from license_plate_detection.evaluation.evaluator import evaluate_license_plate_detection, evaluate_model_comprehensive
from license_plate_detection.evaluation.demo import generate_demo_predictions, create_mock_comprehensive_results
from license_plate_detection.evaluation.error_analysis import analyze_predictions
from license_plate_detection.utils.visualization import visualize_prediction, visualize_processed_sample, plot_training_history as visualize_training_history
from license_plate_detection.utils.analysis import analyze_error_patterns
from license_plate_detection.utils.helpers import detect_license_plate, load_and_prepare_model

# Get paths to dataset using our refactored function
data_path = get_data_path()
IMAGES_PATH = data_path / "images"
ANNOTATIONS_PATH = data_path / "annotations"

# Check if the dataset paths exist
if not data_path.exists():
    raise FileNotFoundError(f"Data path does not exist: {data_path}\n"
                            "Please check the path or create the folder and add your data.")
if not IMAGES_PATH.exists():
    raise FileNotFoundError(f"Images directory does not exist: {IMAGES_PATH}")
if not ANNOTATIONS_PATH.exists():
    raise FileNotFoundError(f"Annotations directory does not exist: {ANNOTATIONS_PATH}")

print(f"Data path: {data_path}")
print(f"Images directory: {IMAGES_PATH}")
print(f"Annotations directory: {ANNOTATIONS_PATH}")

# Load the dataset using the specialized function that returns a DataFrame
df = load_license_plate_dataset(ANNOTATIONS_PATH, IMAGES_PATH)

print(f"Loaded {len(df)} images with annotations")

# Visualize a sample image with its bounding box
if len(df) > 0:
    # Take first sample or specific index if available
    sample_idx = min(1000, len(df) - 1)  # Ensure index exists
    sample = df.iloc[sample_idx]
    img = cv2.imread(sample["image_path"])
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Draw bounding box
    x, y, w, h = sample["x"], sample["y"], sample["w"], sample["h"]
    cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 2)

    plt.figure(figsize=(10, 8))
    plt.imshow(img_rgb)
    if "plate_text" in sample and sample["plate_text"] != "Unknown":
        plt.title(f"Plate: {sample['plate_text']}")
    plt.axis('off')
    plt.show()
    
    # Get statistics on plate sizes to understand the distribution
    df['area'] = df['w'] * df['h']
    df['aspect_ratio'] = df['w'] / df['h']
    
    # Calculate normalized areas relative to image size
    df['norm_area'] = df.apply(lambda row: (row['w'] * row['h']) / 
                             (cv2.imread(row['image_path']).shape[0] * 
                              cv2.imread(row['image_path']).shape[1]), axis=1)
    
    # Plot statistics
    plt.figure(figsize=(15, 5))
    plt.subplot(1, 3, 1)
    plt.hist(df['norm_area'], bins=30)
    plt.title('Normalized Plate Area Distribution')
    plt.xlabel('Normalized Area')
    plt.ylabel('Count')
    
    plt.subplot(1, 3, 2)
    plt.hist(df['aspect_ratio'], bins=30)
    plt.title('Plate Aspect Ratio Distribution')
    plt.xlabel('Aspect Ratio (w/h)')
    
    # Define size categories
    small_threshold = 0.03
    large_threshold = 0.1
    df['size_category'] = 'Medium'
    df.loc[df['norm_area'] < small_threshold, 'size_category'] = 'Small'
    df.loc[df['norm_area'] > large_threshold, 'size_category'] = 'Large'
    
    # Count by size category
    size_counts = df['size_category'].value_counts()
    plt.subplot(1, 3, 3)
    plt.bar(size_counts.index, size_counts.values)
    plt.title('License Plates by Size Category')
    plt.ylabel('Count')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Size distribution:")
    for category in ['Small', 'Medium', 'Large']:
        count = size_counts.get(category, 0)
        percentage = count / len(df) * 100
        print(f"  {category} plates: {count} ({percentage:.1f}%)")
else:
    print("No images loaded. Please check dataset path and XML format.")

# Step 2: Enhanced Data Preprocessing

We'll preprocess our data with a focus on improving size estimation accuracy. 
We'll use a larger input resolution (416×416) to better capture small license plates.

In [ ]:
# Use a larger input resolution for better small plate detection
IMAGE_SIZE = (416, 416)  # Increased from 224x224 for better detail preservation

# Preprocess the dataset
X, y = preprocess_license_plate_dataset(df, image_size=IMAGE_SIZE)

print(f"Processed {len(X)} images to size {IMAGE_SIZE}")
print("Image shape:", X[0].shape)
print("Sample bounding box (normalized):", y[0])

# Show a processed sample
plt.figure(figsize=(8, 8))
plt.imshow(X[0])

# Draw bounding box on sample
h, w = X[0].shape[:2]
box_x = int(y[0][0] * w)
box_y = int(y[0][1] * h)
box_w = int(y[0][2] * w)
box_h = int(y[0][3] * h)

sample_img = X[0].copy()
cv2.rectangle(sample_img, 
              (box_x, box_y), 
              (box_x + box_w, box_y + box_h), 
              (0, 1, 0), 2)

plt.imshow(sample_img)
plt.title("Processed Image with Normalized Bounding Box")
plt.axis('off')
plt.show()

# Analyze bounding box distribution
y_np = np.array(y)
box_widths = y_np[:, 2]
box_heights = y_np[:, 3]
box_areas = box_widths * box_heights
box_aspect_ratios = box_widths / box_heights

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(box_areas, bins=30)
plt.title('Normalized Box Area Distribution')
plt.xlabel('Area')

plt.subplot(1, 3, 2)
plt.hist(box_widths, bins=30, alpha=0.7, label='Width')
plt.hist(box_heights, bins=30, alpha=0.7, label='Height')
plt.title('Box Width and Height Distribution')
plt.legend()

plt.subplot(1, 3, 3)
plt.hist(box_aspect_ratios, bins=30)
plt.title('Box Aspect Ratio Distribution')
plt.xlabel('Width/Height')

plt.tight_layout()
plt.show()

# Calculate statistics for the preprocessed dataset
print("\nPreprocessed dataset statistics:")
print(f"Mean normalized area: {np.mean(box_areas):.4f}")
print(f"Mean aspect ratio: {np.mean(box_aspect_ratios):.4f}")

# Define size categories
small_threshold = 0.03
large_threshold = 0.1
small_count = sum(box_areas < small_threshold)
medium_count = sum((box_areas >= small_threshold) & (box_areas <= large_threshold))
large_count = sum(box_areas > large_threshold)

print("\nSize categories:")
print(f"Small plates (area < {small_threshold}): {small_count} ({small_count/len(box_areas)*100:.1f}%)")
print(f"Medium plates: {medium_count} ({medium_count/len(box_areas)*100:.1f}%)")
print(f"Large plates (area > {large_threshold}): {large_count} ({large_count/len(box_areas)*100:.1f}%)")

# Step 3: Enhanced Data Augmentation

We'll use advanced augmentation techniques with a focus on small plate enhancement and size diversity.

In [ ]:
# Convert to float32 for memory efficiency
X = X.astype(np.float32)
y = y.astype(np.float32)

# Apply enhanced data augmentation with focus on small plates
print("Applying enhanced data augmentation with special focus on small plates...")
X_aug, y_aug = augment_data(X, y, augmentation_factor=5)

print(f"Original dataset size: {len(X)}")
print(f"Augmented dataset size: {len(X_aug)}")
print(f"Augmentation ratio: {len(X_aug) / len(X):.1f}x")

# Visualize augmented samples
visualize_augmentation(X, y, X_aug, y_aug, num_samples=3)

# Analyze augmented bounding box distribution to ensure diversity
y_aug_np = np.array(y_aug)
aug_widths = y_aug_np[:, 2]
aug_heights = y_aug_np[:, 3]
aug_areas = aug_widths * aug_heights
aug_aspect_ratios = aug_widths / aug_heights

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.hist(box_areas, bins=30, alpha=0.7, label='Original')
plt.hist(aug_areas, bins=30, alpha=0.5, label='Augmented')
plt.title('Area Distribution: Original vs Augmented')
plt.xlabel('Area')
plt.legend()

plt.subplot(1, 3, 2)
plt.hist(box_aspect_ratios, bins=30, alpha=0.7, label='Original')
plt.hist(aug_aspect_ratios, bins=30, alpha=0.5, label='Augmented')
plt.title('Aspect Ratio: Original vs Augmented')
plt.xlabel('Width/Height')
plt.legend()

# Count by size category
aug_small_count = sum(aug_areas < small_threshold)
aug_medium_count = sum((aug_areas >= small_threshold) & (aug_areas <= large_threshold))
aug_large_count = sum(aug_areas > large_threshold)

# Plot size category distribution
plt.subplot(1, 3, 3)
categories = ['Small', 'Medium', 'Large']
orig_counts = [small_count, medium_count, large_count]
aug_counts = [aug_small_count, aug_medium_count, aug_large_count]

x = np.arange(len(categories))
width = 0.35

plt.bar(x - width/2, orig_counts, width, label='Original')
plt.bar(x + width/2, aug_counts, width, label='Augmented')
plt.xticks(x, categories)
plt.title('Size Distribution')
plt.ylabel('Count')
plt.legend()

plt.tight_layout()
plt.show()

print("\nAugmented dataset size distribution:")
print(f"Small plates: {aug_small_count} ({aug_small_count/len(aug_areas)*100:.1f}%)")
print(f"Medium plates: {aug_medium_count} ({aug_medium_count/len(aug_areas)*100:.1f}%)")
print(f"Large plates: {aug_large_count} ({aug_large_count/len(aug_areas)*100:.1f}%)")

# Split data into training and validation sets (stratified by plate size)
X_train, X_val, y_train, y_val = split_dataset(X_aug, y_aug, test_size=0.2, random_state=42)
print(f"Training samples: {len(X_train)}, Validation samples: {len(X_val)}")

# Step 4: Create Enhanced FPN Architecture

We'll create our improved Feature Pyramid Network architecture with special focus on size estimation and small plate detection.

In [ ]:
# Create the improved FPN detector with 416×416 input size
print("Creating enhanced FPN license plate detector...")
fpn_model = create_fpn_license_plate_detector(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))

# Display model summary
fpn_model.summary()

# Print model size information
trainable_count = np.sum([keras.backend.count_params(w) for w in fpn_model.trainable_weights])
non_trainable_count = np.sum([keras.backend.count_params(w) for w in fpn_model.non_trainable_weights])
total_params = trainable_count + non_trainable_count

print(f"\nModel Size Information:")
print(f"Total parameters: {total_params:,}")
print(f"Trainable parameters: {trainable_count:,} ({trainable_count/total_params*100:.1f}%)")
print(f"Non-trainable parameters: {non_trainable_count:,} ({non_trainable_count/total_params*100:.1f}%)")

# Visualize model architecture (optional)
try:
    from tensorflow.keras.utils import plot_model
    plot_model(fpn_model, to_file='fpn_model.png', show_shapes=True, show_layer_names=True)
    from IPython.display import Image
    Image('fpn_model.png')
except:
    print("Could not visualize model architecture. Missing dependencies.")

# Step 5: GPU and Memory Optimizations

We'll apply memory optimizations to enable training of our model on available hardware.

In [ ]:
# Configure GPU to grow memory as needed
setup_gpu_memory_growth()

# Limit GPU memory if needed
# Uncomment and set appropriate value if you face memory issues
# GPU_MEMORY_LIMIT_MB = 11 * 1024  # 11GB
# limit_gpu_memory(GPU_MEMORY_LIMIT_MB)

# Enable mixed precision training
try:
    if tf.__version__.startswith('2'):
        policy = tf.keras.mixed_precision.Policy('mixed_float16')
        tf.keras.mixed_precision.set_global_policy(policy)
        print("Mixed precision enabled:")
        print(f"  Compute dtype: {policy.compute_dtype}")
        print(f"  Variable dtype: {policy.variable_dtype}")
    else:
        print("Mixed precision only available in TF 2.x")
except Exception as e:
    print(f"Could not enable mixed precision: {e}")

# Clean up memory
clean_memory()

# Optimize training data formats
X_train, X_val, y_train, y_val = optimize_memory_usage(X_train, y_train, X_val, y_val)

# Apply gradient checkpointing to reduce memory usage
print("Applying gradient checkpointing...")
fpn_model = enable_gradient_checkpointing(fpn_model)

print("Memory optimizations complete.")

# Step 6: Model Training with Enhanced Strategy

We'll use our improved combined loss function with cosine learning rate schedule and extended patience for early stopping.

In [ ]:
# Compile the model with our improved loss function
fpn_model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss=improved_combined_detection_loss,
    metrics=[enhanced_iou_metric, giou_loss]
)

# Create callbacks with optimized parameters
callbacks = [
    # Cosine learning rate scheduler with warmup
    create_lr_scheduler(
        scheduler_type='cosine',
        initial_learning_rate=0.001,
        epochs=100,  # Increased epochs
        warmup_epochs=10  # Extended warmup period
    ),
    
    # Early stopping with extended patience
    keras.callbacks.EarlyStopping(
        monitor='val_enhanced_iou_metric',
        patience=30,  # Extended patience from 20 to 30
        restore_best_weights=True,
        mode='max'
    ),
    
    # Model checkpoint
    keras.callbacks.ModelCheckpoint(
        'improved_fpn_license_plate_detector.h5',
        monitor='val_enhanced_iou_metric',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Reduce learning rate on plateau as a backup strategy
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=0.00001,
        verbose=1
    ),
    
    # TensorBoard logging
    keras.callbacks.TensorBoard(
        log_dir='./logs/fpn_model',
        histogram_freq=1,
        write_graph=True
    )
]

# Create efficient data pipeline
BATCH_SIZE = 16
print(f"Creating optimized data pipeline with batch size {BATCH_SIZE}...")
train_dataset, val_dataset = create_efficient_data_pipeline(
    X_train, y_train, X_val, y_val,
    batch_size=BATCH_SIZE
)

# Add on-the-fly data augmentation
def additional_augment(image, bbox):
    """Additional on-the-fly augmentation to improve generalization"""
    # Random brightness
    image = tf.image.random_brightness(image, max_delta=0.1)
    # Random contrast
    image = tf.image.random_contrast(image, lower=0.9, upper=1.1)
    # Random saturation
    image = tf.image.random_saturation(image, lower=0.9, upper=1.1)
    # Ensure values remain in [0,1]
    image = tf.clip_by_value(image, 0.0, 1.0)
    return image, bbox

# Apply on-the-fly augmentation
augmented_train_dataset = train_dataset.map(
    additional_augment,
    num_parallel_calls=tf.data.experimental.AUTOTUNE
)

# Print training summary
print("\nTraining Configuration Summary:")
print(f"Input shape: {IMAGE_SIZE[0]}×{IMAGE_SIZE[1]}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Training samples: {len(X_train)}")
print(f"Validation samples: {len(X_val)}")
print(f"Initial learning rate: 0.001")
print(f"Loss function: improved_combined_detection_loss")
print(f"Architecture: Feature Pyramid Network with Attention Mechanisms")

print("\nStarting model training...")
history = fpn_model.fit(
    augmented_train_dataset,
    validation_data=val_dataset,
    epochs=100,  # Maximum epochs
    callbacks=callbacks,
    verbose=1
)

# Save the final model
trained_fpn_model = fpn_model
save_model(trained_fpn_model, 'improved_fpn_license_plate_detector_final.h5')
print("Model saved to 'improved_fpn_license_plate_detector_final.h5'")

# Visualize training history
visualize_training_history(history)

# Step 7: Comprehensive Model Evaluation

We'll evaluate our trained model with detailed metrics and analysis.

In [ ]:
# Generate predictions with the trained model
print("Generating predictions for evaluation...")
y_pred = trained_fpn_model.predict(X_val)

# Run comprehensive evaluation
print("\nRunning comprehensive evaluation...")
evaluation_results = evaluate_model_comprehensive(
    trained_fpn_model,
    X_val,
    y_val,
    y_pred
)

# Print key metrics
print("\n===== EVALUATION RESULTS =====")
print("Overall Performance:")
print(f"  Mean IoU: {evaluation_results['mean_iou']:.4f}")
print(f"  Median IoU: {evaluation_results['median_iou']:.4f}")
print(f"  mAP@0.5: {evaluation_results['map50']:.4f}")
print(f"  mAP@0.5:0.95: {evaluation_results['map']:.4f}")

print("\nPerformance by Plate Size:")
for size in ["small", "medium", "large"]:
    if f'{size}_count' in evaluation_results:
        count = evaluation_results[f'{size}_count']
        mean_iou = evaluation_results[f'{size}_mean_iou']
        map50 = evaluation_results[f'{size}_map50']
        print(f"  {size.capitalize()} Plates: Count={count}, Mean IoU={mean_iou:.4f}, mAP@0.5={map50:.4f}")

print("\nCoordinate Errors (Normalized):")
print(f"  Center Point Error: {evaluation_results['mean_center_error']:.4f}")
print(f"  Size Error: {evaluation_results['mean_size_error']:.4f}")
print(f"  X Error: {evaluation_results['mean_x_error']:.4f}, Y Error: {evaluation_results['mean_y_error']:.4f}")
print(f"  Width Error: {evaluation_results['mean_width_error']:.4f}, Height Error: {evaluation_results['mean_height_error']:.4f}")

# Visualize some predictions
print("\nVisualizing sample predictions...")
num_samples = 4
indices = np.random.randint(0, len(X_val), num_samples)

plt.figure(figsize=(15, num_samples * 4))
for i, idx in enumerate(indices):
    img = X_val[idx]
    true_box = y_val[idx]
    pred_box = y_pred[idx]
    
    # Calculate IoU
    iou = enhanced_iou_metric(np.expand_dims(true_box, 0), np.expand_dims(pred_box, 0))
    
    # Draw both boxes on image
    img_with_boxes = img.copy()
    h, w = img.shape[:2]
    
    # Draw ground truth box (green)
    x1 = int(true_box[0] * w - true_box[2] * w / 2)
    y1 = int(true_box[1] * h - true_box[3] * h / 2)
    x2 = int(true_box[0] * w + true_box[2] * w / 2)
    y2 = int(true_box[1] * h + true_box[3] * h / 2)
    cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (0, 1, 0), 2)
    
    # Draw predicted box (red)
    x1 = int(pred_box[0] * w - pred_box[2] * w / 2)
    y1 = int(pred_box[1] * h - pred_box[3] * h / 2)
    x2 = int(pred_box[0] * w + pred_box[2] * w / 2)
    y2 = int(pred_box[1] * h + pred_box[3] * h / 2)
    cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (1, 0, 0), 2)
    
    # Calculate plate size category
    plate_area = true_box[2] * true_box[3]
    if plate_area < small_threshold:
        size_category = "Small"
    elif plate_area > large_threshold:
        size_category = "Large"
    else:
        size_category = "Medium"
    
    plt.subplot(num_samples, 1, i+1)
    plt.imshow(img_with_boxes)
    plt.title(f"Sample {idx}: IoU={iou.numpy()[0]:.4f}, Size={size_category}\n" +
              f"True: {true_box}\nPred: {pred_box}")
    plt.axis('off')
    
plt.tight_layout()
plt.show()

# Step 8: Detailed Error Analysis

We'll perform a detailed error analysis to understand remaining issues.

In [ ]:
# Perform detailed error analysis
print("Performing detailed error analysis...")
error_analysis = analyze_error_patterns(
    model=trained_fpn_model,
    X_val=X_val,
    y_val=y_val,
    y_pred=y_pred,
    plate_sizes=[box[2] * box[3] for box in y_val]
)

# Print summary statistics from the error analysis
print("\n===== ERROR ANALYSIS SUMMARY =====")
if 'metrics' in error_analysis:
    metrics = error_analysis['metrics']
    print(f"Overall mean IoU: {metrics['mean_iou']:.4f}")
    print(f"IoU standard deviation: {metrics['iou_std']:.4f}")
    
    if 'error_correlation' in metrics:
        print("\nError Correlations:")
        for key, value in metrics['error_correlation'].items():
            print(f"  {key}: {value:.4f}")
    
    if 'size_metrics' in metrics:
        print("\nPerformance by Size Category:")
        for size, size_metrics in metrics['size_metrics'].items():
            print(f"  {size} plates:")
            print(f"    Count: {size_metrics['count']}")
            print(f"    Mean IoU: {size_metrics['mean_iou']:.4f}")
            print(f"    Position error: {size_metrics['position_error']:.4f}")
            print(f"    Size error: {size_metrics['size_error']:.4f}")
    
    if 'recommendations' in error_analysis:
        print("\nRecommendations for Further Improvement:")
        for i, rec in enumerate(error_analysis['recommendations']):
            print(f"  {i+1}. {rec}")

# Print comparison with previous results
print("\n===== IMPROVEMENT OVER PREVIOUS MODEL =====")
previous_iou = 0.2514  # From previous evaluation
current_iou = evaluation_results['mean_iou']
improvement = (current_iou - previous_iou) / previous_iou * 100
print(f"Previous model mean IoU: {previous_iou:.4f}")
print(f"Current model mean IoU: {current_iou:.4f}")
print(f"Improvement: {improvement:.1f}%")

# Check if target IoU was achieved
target_iou = 0.6
if current_iou >= target_iou:
    print(f"\nSUCCESS! Target IoU of {target_iou} was achieved.")
else:
    print(f"\nTarget IoU of {target_iou} not yet achieved. Current IoU: {current_iou:.4f}")
    print("Additional improvements may be needed.")

# Step 9: License Plate Detection Demo

Let's test our model on a few sample images to see how it performs.

In [ ]:
# Test the model on sample images
if len(df) > 0:
    # Choose a few test images
    test_indices = np.random.choice(len(df), 3)
    
    for i, idx in enumerate(test_indices):
        # Get image path
        img_path = df.iloc[idx]["image_path"]
        
        # Detect license plate
        print(f"\nDetecting license plate in image {i+1}...")
        
        # Load and preprocess the image
        img = cv2.imread(img_path)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        orig_h, orig_w = img_rgb.shape[:2]
        
        # Resize to model input size
        img_resized = cv2.resize(img_rgb, IMAGE_SIZE)
        img_normalized = img_resized.astype(np.float32) / 255.0
        
        # Make prediction
        pred = trained_fpn_model.predict(np.expand_dims(img_normalized, 0))[0]
        
        # Convert normalized coordinates to original image coordinates
        # Bounding box format: [x_center, y_center, width, height]
        x_center = pred[0] * orig_w
        y_center = pred[1] * orig_h
        width = pred[2] * orig_w
        height = pred[3] * orig_h
        
        # Calculate box corners
        x1 = int(x_center - width / 2)
        y1 = int(y_center - height / 2)
        x2 = int(x_center + width / 2)
        y2 = int(y_center + height / 2)
        
        # Ensure coordinates are within image bounds
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(orig_w, x2)
        y2 = min(orig_h, y2)
        
        # Draw prediction on image
        result_img = img_rgb.copy()
        cv2.rectangle(result_img, (x1, y1), (x2, y2), (0, 255, 0), 3)
        
        # Get ground truth
        gt_x, gt_y, gt_w, gt_h = df.iloc[idx]["x"], df.iloc[idx]["y"], df.iloc[idx]["w"], df.iloc[idx]["h"]
        
        # Draw ground truth on image (red)
        cv2.rectangle(result_img, (gt_x, gt_y), (gt_x + gt_w, gt_y + gt_h), (255, 0, 0), 2)
        
        # Calculate IoU
        def calculate_iou(box1, box2):
            """Calculate IoU between two boxes in format [x1, y1, x2, y2]"""
            x1 = max(box1[0], box2[0])
            y1 = max(box1[1], box2[1])
            x2 = min(box1[2], box2[2])
            y2 = min(box1[3], box2[3])
            
            intersection = max(0, x2 - x1) * max(0, y2 - y1)
            box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
            box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
            union = box1_area + box2_area - intersection
            
            return intersection / union if union > 0 else 0
        
        pred_box = [x1, y1, x2, y2]
        gt_box = [gt_x, gt_y, gt_x + gt_w, gt_y + gt_h]
        iou = calculate_iou(pred_box, gt_box)
        
        # Extract plate region
        plate_region = img_rgb[y1:y2, x1:x2]
        
        # Display result
        plt.figure(figsize=(12, 8))
        plt.imshow(result_img)
        plt.title(f"License Plate Detection - IoU: {iou:.4f}\n" +
                 f"Green: Prediction, Red: Ground Truth")
        plt.axis('off')
        plt.show()
        
        # Show extracted plate
        if plate_region.size > 0:
            plt.figure(figsize=(6, 3))
            plt.imshow(plate_region)
            plt.title("Extracted License Plate")
            plt.axis('off')
            plt.show()
        else:
            print("Invalid detection - no plate extracted")
else:
    print("No images available for demonstration")

# Conclusion and Further Recommendations

Our improved FPN model significantly enhances license plate detection accuracy, particularly for size estimation and small plate detection. Key improvements include:

1. **Enhanced Architecture**: Feature Pyramid Network with multi-scale capabilities
2. **Attention Mechanisms**: Channel and spatial attention for focusing on relevant features
3. **Improved Loss Function**: Better balancing of size, position, and IoU components
4. **Advanced Data Augmentation**: Special focus on small plate augmentation
5. **Higher Resolution**: 416×416 input size for better detail preservation

To further improve performance beyond what we've achieved:

1. **Ensemble Methods**: Combine multiple models trained with different architectures
2. **Test-Time Augmentation**: Apply multiple transformations during inference and average results
3. **Two-Stage Detection**: Add a refinement stage focused exclusively on accurate size estimation
4. **Additional Training Data**: Collect more examples of problematic cases (small plates, unusual angles)
5. **Domain-Specific Post-Processing**: Apply prior knowledge about license plate aspect ratios